In [ ]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

warnings.filterwarnings("ignore")
%matplotlib inline

**Data preparation**

In [ ]:
# read dataset
def read_data(filename):
    data = np.loadtxt(filename, delimiter=',')
    x = data[:, 1:]
    y = data[:, 0]

    return (x, y)


file_name = '/content/drive/MyDrive/data.txt'
X, y = read_data(file_name)

print(f'X:shape = {X.shape}, y:shape = {y.shape}')

In [ ]:
# plot every class time series
fig, axs = plt.subplots(nrows=7, ncols=2, figsize=(15, 10),
                        subplot_kw={'xticks': [], 'yticks': []})

for ax, n in zip(axs.flat, range(0, X.shape[0], 9)):
    ax.plot(X[n, :])
    ax.set_title(f'Time series: {n}')

In [ ]:
# count unique classes
num_class = len(np.unique(y))

print(f'Number of classes: {num_class}')

In [ ]:
# train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42, shuffle=True)

In [ ]:
# scale the data
scaler = StandardScaler()

X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

print(f'X_train:shape = {X_train.shape}, X_test:shape = {X_test.shape}')

In [ ]:
# encoding class labels
encoder = OneHotEncoder(categories='auto')

Y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
Y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

print(f'Y_train:shape = {Y_train.shape}, Y_test:shape = {Y_test.shape}')

**Machine learning**

In [ ]:
# run each classfier
def run(x_train, y_train, x_test, y_test, clf):
    clf.fit(x_train, y_train)
    score = 100 * clf.score(x_test, y_test)
    print("score = %0.2f%%" % score)


# K-Neighbors and Random Forest classifiers
def first_set(x_train, y_train, x_test, y_test):
    print("k-NN classifier (k=3) : ", end='')
    run(x_train, y_train, x_test, y_test, KNeighborsClassifier(n_neighbors=3))

    print("k-NN classifier (k=7) : ", end='')
    run(x_train, y_train, x_test, y_test, KNeighborsClassifier(n_neighbors=7))

    print("Random Forest (trees= 5) : ", end='')
    run(x_train, y_train, x_test, y_test, RandomForestClassifier(n_estimators=5))

    print("Random Forest (trees= 50) : ", end='')
    run(x_train, y_train, x_test, y_test,
        RandomForestClassifier(n_estimators=50))

    print("Random Forest (trees=500) : ", end='')
    run(x_train, y_train, x_test, y_test,
        RandomForestClassifier(n_estimators=500))

    print("Random Forest (trees=1000): ", end='')
    run(x_train, y_train, x_test, y_test,
        RandomForestClassifier(n_estimators=1000))


# Nearest Centroid, Naive Bayes and Linear SVM classifiers
def second_set(x_train, y_train, x_test, y_test):
    print("Nearest Centroid : ", end='')
    run(x_train, y_train, x_test, y_test, NearestCentroid())

    print("Naive Bayes (Gaussian) : ", end='')
    run(x_train, y_train, x_test, y_test, GaussianNB())

    print("LinearSVM (C=0.01) : ", end='')
    run(x_train, y_train, x_test, y_test, LinearSVC(C=0.01))

    print("LinearSVM (C=0.1) : ", end='')
    run(x_train, y_train, x_test, y_test, LinearSVC(C=0.1))

    print("LinearSVM (C=1.0) : ", end='')
    run(x_train, y_train, x_test, y_test, LinearSVC(C=1.0))

    print("LinearSVM (C=10.0) : ", end='')
    run(x_train, y_train, x_test, y_test, LinearSVC(C=10.0))


first_set(X_train, Y_train, X_test, Y_test)
second_set(X_train, y_train, X_test, y_test)

**Deep learning**

In [ ]:
# Multilayer Perceptron (MLP)
class MLP:
    def __init__(self, input_shape, num_class, verbose, build):
        self.input_shape = input_shape
        self.num_class = num_class
        self.verbose = verbose
        if build:
            self.model = self.build_model()

        self.model.summary()

    def build_model(self):
        model = Sequential()
        model.add(Dense(1024, activation='relu', input_shape=self.input_shape))
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(num_class, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

        return model

    def fit(self, num_epochs, x_train, y_train, x_val, y_val):
        batch_size = 32

        history = self.model.fit(x_train, y_train, batch_size=batch_size,
                                 epochs=num_epochs, verbose=self.verbose,
                                 validation_data=(x_val, y_val))

        self.score = self.model.evaluate(X_test, Y_test, verbose=0)
        print('Test accuracy:', self.score[1])

        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

    def predict(self, x):
        y_pred = self.model.predict(x)

        return y_pred

In [ ]:
input_shape = X_train.shape[1:]

classifier_mlp = MLP(input_shape, num_class, False, True)

In [ ]:
classifier_mlp.fit(500, X_train, Y_train, X_test, Y_test)

In [ ]:
# Shallow Network
class ShNet:
    def __init__(self, input_shape: tuple, num_class: int, verbose: bool, build: bool) -> None:
        self.input_shape = input_shape
        self.num_class = num_class
        self.verbose = verbose
        if build:
            self.model = self.build_model()

        self.model.summary()

    def build_model(self):
        model = Sequential()
        model.add(Conv1D(32, kernel_size=3, activation='relu',
                  input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=3))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_class, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        return model

    def fit(self, num_epochs: int, x_train: tuple, y_train: tuple, x_val: tuple, y_val: tuple) -> None:
        batch_size = 32

        history = self.model.fit(x_train, y_train, batch_size=batch_size,
                                 epochs=num_epochs, verbose=self.verbose,
                                 validation_data=(x_val, y_val))

        self.score = self.model.evaluate(X_test, Y_test, verbose=0)
        print('Test accuracy:', self.score[1])

        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

    def predict(self, x: np.array) -> np.array:
        y_pred = self.model.predict(x)

        return y_pred

In [ ]:
# add a dimension to make it multivariate
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
input_shape = X_train.shape[1:]
classifier_fcn = ShNet(input_shape, num_class, False, True)

In [ ]:
classifier_fcn.fit(2000, X_train, Y_train, X_test, Y_test)